Copyright (C) 2021-2022, Intel Corporation

SPDX-License-Identifier: Apache-2.0

Major Portions of this code are copyright of their respective authors and released under the General Public License Version 3.0:
- For licensing see https://github.com/WongKinYiu/yolov7/blob/main/LICENSE.md

# Object detection with YOLOv7 in Python using OpenVINO™ Execution Provider:

1. The Object detection sample uses a YOLOv7 Deep Learning ONNX Model.


2. The sample involves presenting an image to ONNX Runtime (RT), which uses the OpenVINO™ Execution Provider to run inference on various Intel hardware devices as mentioned before and perform object detection to detect up to 80 different objects like person, bicycle, car, motorbike and much more from the coco dataset.

**Before starting with this notebook please make sure to perform the required installations as mentioned below:**
1. [YoloV7 installation requirements](https://github.com/WongKinYiu/yolov7#installation)
2. [NNCF Onnx (Experimental Requirements)](https://github.com/openvinotoolkit/nncf/tree/54a444fb6ef1806c7ab8e647f762e2547ceb95bf#installation)

## Download COCO validation dataset

In [ ]:
import os
import sys
import warnings
from urllib import request
from zipfile import ZipFile
import torch

warnings.filterwarnings(action="ignore")
operating_platform = sys.platform

# Download COCO val
if not os.path.exists("datasets"):
    torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
    if operating_platform == "win32":
        with ZipFile("tmp.zip", "r") as zipped_data:
            zipped_data.extractall(path="datasets")
        zipped_data.close()
        os.remove("tmp.zip")
    else:
        !unzip -q tmp.zip -d datasets && rm tmp.zip


## Export yolov7 model to ONNX format with NMS
**Note: Adjust the values of `iou-thres` and `conf-thres` according to requirements.**

In [ ]:
if not os.path.exists("yolov7.onnx"):
    if operating_platform == "win32":
        model_url = "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt"
        request.urlretrieve(model_url, "yolov7.pt")
    else:
        !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
    
    !python export.py --weights yolov7.pt --grid --end2end --simplify --topk-all 100 --iou-thres 0.65 --conf-thres 0.35 --img-size 640 640 --max-wh 640

In [ ]:
if not os.path.exists('cat.jpg'):
    if operating_platform == "win32":
        img_url = "https://storage.openvinotoolkit.org/data/test_data/images/cat.jpg"
        request.urlretrieve(img_url, "cat.jpg")
    else:
        !wget https://storage.openvinotoolkit.org/data/test_data/images/cat.jpg

## Import Required Libraries

In [ ]:
# Inference for ONNX model
import cv2
import time
import requests
import random
import logging
import numpy as np
import onnxruntime
from PIL import Image
from pathlib import Path
from collections import OrderedDict,namedtuple
import matplotlib.pyplot as plt

## Define necessary helper functions
### Pre-Processing

When we are using a pre-trained model, which is trained & fine-tuned using a fixed image size as input, we should resize our image to a shape which is expected by the model. The image reshaped using a scaling factor which is a ratio between the desired height/width and the actual image height/width.  
$$scale = min \biggl( \frac{\text{target height}}{\text{input image height}}, \frac{\text{target width}}{\text{input image width}} \biggl)$$  
Using the $scale$-factor, image height & width are calculated which is then re-shaped to the desired image size using the `opencv` package. Here this is acheived by the `preProcess_image` helper function.

In [ ]:
def letterbox(im, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better val mAP)
        r = min(r, 1.0)

    # Compute padding
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding

    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return im, r, (dw, dh)

## Labels and Colors for Lables

In [ ]:
random.seed(42)
names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 
         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 
         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 
         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 
         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 
         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 
         'hair drier', 'toothbrush']
colors = {name:[random.randint(0, 255) for _ in range(3)] for i,name in enumerate(names)}

## Read & Pre-Process Image

In [ ]:
def set_logging(rank=-1):
    logging.basicConfig(
        format="%(message)s",
        level=logging.INFO if rank in [-1, 0] else logging.WARN)
    
set_logging(0)  # run before defining LOGGER
LOGGER = logging.getLogger("yolov7")

def preProcess_image(img_path):
    

    if img_path.split('.')[-1] not in ['jpg', 'png']:
        raise Exception(img_path, "is an unknown file format. Use the image of .jpg or .png format")
    
    if not os.path.exists(img_path):
        raise FileNotFoundError(img_path, "File Not Found: Please enter a valid image file path!")

    if os.path.islink(img_path):
        raise Exception(img_path, "is a symbolic link. Please provide a valid image file path!")

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    image = img.copy()
    image, ratio, dwdh = letterbox(image, auto=False)
    image = image.transpose((2, 0, 1))
    image = np.expand_dims(image, 0)
    image = np.ascontiguousarray(image)

    im = image.astype(np.float32)
    im /= 255
    print("Image Shape:", im.shape, sep='\t')
    return im, ratio, dwdh

## Create a session for inference based on the device selected
Inferencing using OpenVINO Execution provider under ONNX-Runtime, is performed using the below simple steps:

1. Create a ONNX Runtime Inference Session instance using `onnxruntime.InferenceSession()`
2. To create an Inference Session object pass the model and the execution provider as arguments. Execution Providers are the hardware device options e.g. CPU, Myriad, GPU, etc. on which the session will be executed.

The below create_sess function actually takes care of the above steps. All we need to do is pass the device arguement to it. It'll return the appropriate session according to the selected device along with the input name for the model.

In [ ]:
def check_model_extension(fp):
  # Split the extension from the path and normalise it to lowercase.
  ext = os.path.splitext(fp)[-1].lower()

  # Now we can simply use != to check for inequality, no need for wildcards.
  if(ext != ".onnx"):
    raise Exception(fp, "is an unknown file format. Use the model ending with .onnx format")
  
  if not os.path.exists(fp):
    raise Exception("[ ERROR ] Path of the onnx model file is Invalid")
  
  if os.path.islink(fp):
        raise Exception(fp, "is a symbolic link. Please provide a valid .onnx model path!")

def create_session(model_path, device='CPU_FP32'):
    
    check_model_extension(model_path)

    if device == 'CPU_FP32':
        providers = ['OpenVINOExecutionProvider']
    elif device == 'cpu':
        providers = ['CPUExecutionProvider']
    else:
        print('Invalid or No provider passed, using default CPU Execution Provider.')
        providers = ['CPUExecutionProvider']
    
    print(f'Use ORT providers: {providers}')
    onnxruntime.set_default_logger_severity(4)

    sess = onnxruntime.InferenceSession(model_path,
                                        providers=providers,
                                        provider_options=[{'device_type': device}])

    outname = [i.name for i in sess.get_outputs()]
    inname = [i.name for i in sess.get_inputs()]
    
    return sess, outname, inname


# Run Inference Function

The `run-inference` function takes inputs as:
- `model_path` $\rightarrow$ `<path to the model>.onnx`
- `img_input` $\rightarrow$ `<path to the image>.jpg`
- `device` $\rightarrow$ `device using which the inference should be run e.g. cpu, CPU_FP32`  
    Based on the device arguement the execution providers are selected.

It performs the following tasks:
- Pre-process the input image as per model requirements
- Creates the appropriate onnxruntime session as per the device arguement
- Runs inferences along with non-max suppression on the predictions for the number of runs passed
- Adds bounding boxes on the detected objects
- Saves a copy of the inferred image.

In [ ]:
def run_inference(model_path, img_input, device='CPU_FP32', num_runs=10, warm_up=3):
        
    img0, proces_ratio, dwdh = preProcess_image(img_input)
    session, outputs, inputs = create_session(model_path, device)
    
    pred_times = []
    inp = {inputs[0]:img0}

    
    # Inference
    for iter_num in range(num_runs + 2):

        # warmup session
        if iter_num <= warm_up:
            session.run(outputs, inp)[0]
            continue

        start = time.time()
        pred = session.run(outputs, inp)[0]
        end = time.time()
        inference_time = end - start
        pred_times.append(inference_time)
        

    ori_images = cv2.imread(img_input)
    ori_images = [cv2.cvtColor(ori_images, cv2.COLOR_BGR2RGB)]
    
    # Add bounding box to detected samples
    for i,(batch_id,x0,y0,x1,y1,cls_id,score) in enumerate(pred):
        image = ori_images[int(batch_id)]
        box = np.array([x0,y0,x1,y1])
        box -= np.array(dwdh*2)
        box /= proces_ratio
        box = box.round().astype(np.int32).tolist()
        cls_id = int(cls_id)
        score = round(float(score),3)
        name = names[cls_id]
        color = colors[name]
        name += ' '+str(score)
        cv2.rectangle(image,box[:2],box[2:],color,2)
        cv2.putText(image,name,(box[0], box[1] - 2),cv2.FONT_HERSHEY_SIMPLEX,0.45,[225, 255, 255],thickness=1)
        cv2.putText(image, 'FPS: {:.8f}'.format(1.0 / inference_time),
                                  (10, 40), cv2.FONT_HERSHEY_COMPLEX, 0.45, (255, 255, 255), 0) 
    print('Avg Inference time in ms: %f' %
          (sum(pred_times) / len(pred_times) * 1000))
    plt.imshow(Image.fromarray(ori_images[0]))
    plt.axis('off')
    f = f"{img_input.split('.')[0]}_{model_path.split('.')[0]}_{device}.jpg"
    plt.imsave(f, Image.fromarray(ori_images[0]))

### Inference using CPU Execution Provider (MLAS)

In [ ]:
model_path = 'yolov7.onnx'
img_path = 'cat.jpg'
num_iters = 100

In [ ]:
params = {"model_path":model_path,
          "img_input": img_path,
          "device":'cpu',
          "num_runs": num_iters}

run_inference(**params)

### Inference using OpenVino Execution Provider

In [ ]:
params = {"model_path": model_path,
          "img_input": img_path,
          "device":'CPU_FP32',
          "num_runs": num_iters}

run_inference(**params)

# NNCF PTQ for YoloV7

## Quantize yolov7-tiny model

### Build PTQ API dataset

In [ ]:
import onnx
from nncf.experimental.post_training.compression_builder import CompressionBuilder
from nncf.experimental.post_training.algorithms.quantization import PostTrainingQuantization
from nncf.experimental.post_training.algorithms.quantization import PostTrainingQuantizationParameters
from nncf.common.utils.logger import logger as nncf_logger
from nncf.experimental.post_training.api import dataset as ptq_api_dataset
from nncf.experimental.onnx.tensor import ONNXNNCFTensor
from utils.datasets import LoadImagesAndLabels

class YoloV7Dataset(ptq_api_dataset.Dataset):
    def __init__(self, path, batch_size, shuffle):
        super().__init__(batch_size, shuffle)
        self.load_images = LoadImagesAndLabels(path)

    def __getitem__(self, item):
        img, _, _, _ = self.load_images[item]
        # Input should be in [0,1].
        img = (1 / 255.) * img
        return {"images": ONNXNNCFTensor(img.numpy())}

    def __len__(self):
        return len(self.load_images)

dataset = YoloV7Dataset("datasets/coco/images/val2017", 1, True)

### Run PTQ

In [ ]:
original_model = onnx.load("yolov7-tiny.onnx")
num_init_samples = 100
# We'll ignore detector head not to quantize them
ignored_scopes = [
    # Head branch 1
    "Mul_217",
    "Add_219",
    "Mul_221",
    "Mul_223",
    "Mul_227",
    # Head branch 2
    "Mul_251",
    "Add_253",
    "Mul_255",
    "Mul_257",
    "Mul_261",
    # Head branch 3
    "Mul_285",
    "Add_287",
    "Mul_289",
    "Mul_291",
    "Mul_295",
]
output_model_path = "yolov7-tiny-quantized.onnx"

### Step 1: Create a pipeline of compression algorithms.

In [ ]:
builder = CompressionBuilder()

### Step 2: Create the quantization algorithm and add to the builder.

In [ ]:
quantization_parameters = PostTrainingQuantizationParameters(
    number_samples=num_init_samples,
    ignored_scopes=ignored_scopes
)
quantization = PostTrainingQuantization(quantization_parameters)
builder.add_algorithm(quantization)

### Step 3: Execute the pipeline.

In [ ]:
nncf_logger.info("Post-Training Quantization has just started!")
nncf_logger.setLevel(4)
quantized_model = builder.apply(original_model, dataset)

### Step 4: Save the quantized model.

In [ ]:
onnx.save(quantized_model, output_model_path)
nncf_logger.info(
    "The quantized model is saved on {}".format(output_model_path))

onnx.checker.check_model(output_model_path)

## Run Inference on INT-8 model

In [ ]:
model_path = 'yolov7-tiny-quantized.onnx'
img_path = 'cat.jpg'
num_iters = 100

### Inference using OpenVino Execution Provider

In [ ]:
params = {"model_path": model_path,
          "img_input": img_path,
          "device":'CPU_FP32',
          "num_runs": num_iters}

run_inference(**params)

# Benchmarking

- Benchmarks can be performed using:
    - openvino's [benchmark app](https://github.com/openvinotoolkit/openvino/tree/master/tools/benchmark_tool)
```
./benchmark_app -m <path yo model>/yolov7-tiny.onnx  --shape [1,3,640,640]
```
    
<!-- - `onnxruntime_perf_test` tool -->


<!-- https://github.com/openvinotoolkit/open_model_zoo/tree/master/tools/accuracy_checker
accuracy_check -c path/to/configuration_file -m /path/to/models -s /path/to/source/data -a /path/to/annotation -->